In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from xml.dom import minidom
from layer import Grain, Layer
from snowProfile import SnowProfile, SurfaceCondition, Layer, TempMeasurement, WeatherConditions
from whumpfData import WumphData

## Try other parser

In [2]:
import xml.etree.ElementTree as ET
#file_path = "snowpits/wumph_pits/snowpits-25968-caaml.xml"
file_path = "snowpits/mkc_TESTPIT-23-Feb.caaml"


common_tag = '{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}'
root = ET.parse(file_path).getroot()


In [3]:
# measurementDirection
try:
    measurementDirection = next(root.iter(common_tag + 'SnowProfileMeasurements'), None).get('dir')
except AttributeError:
    measurementDirection = None

print(measurementDirection)


top down


In [4]:
#user = {
 #           'OperationID': None,
  #          'OperationName': None,
   #         'Professional': None,
    #        'ContactPersonID': None,
     #       'ContactPersonName': None,
      #      }   

user = {}


try:
    srcRef = root.iter(common_tag + 'srcRef')
except AttributeError:
    srcRef = None

print(srcRef)

if srcRef is not None:
    for prop in srcRef:
        print("tag: ",prop.tag)
        print("attrib: ",prop.attrib)
        print("text: ",prop.text)
        for sub_prop in prop:
            print("sub_tag: ",sub_prop.tag)
            print("sub_attrib: ",sub_prop.attrib)
            print("sub_text: ",sub_prop.text)
            for sub_sub_prop in sub_prop:
                print("sub_sub_tag: ",sub_sub_prop.tag)
                print("sub_sub_attrib: ",sub_sub_prop.attrib)
                print("sub_sub_text: ",sub_sub_prop.text)
                for sub_sub_sub_prop in sub_sub_prop:
                    print("sub_sub_sub_tag: ",sub_sub_sub_prop.tag)
                    print("sub_sub_sub_attrib: ",sub_sub_sub_prop.attrib)
                    print("sub_sub_sub_text: ",sub_sub_sub_prop.text)

if srcRef is not None:
    for prop in srcRef:
        for sub_prop in prop:
            if sub_prop.tag.endswith('Operation'):
                user['OperationID'] = sub_prop.attrib['gml:id']
                for sub_sub_prop in sub_prop:
                    if sub_sub_prop.tag.endswith('name'):
                        user['OperationName'] = sub_sub_prop.text
                    if sub_sub_prop.tag.endswith('contactPerson'):
                        user['ContactPersonID'] = sub_sub_prop.attrib['gml:id']
                        for sub_sub_sub_prop in sub_sub_prop:
                            if sub_sub_sub_prop.tag.endswith('name'):
                                user['ContactPersonName'] = sub_sub_sub_prop.text
            elif sub_prop.tag.endswith('Person'):
                user['ContactPersonID'] = sub_prop.attrib['gml:id']
                for sub_sub_prop in sub_prop:
                    if sub_sub_prop.tag.endswith('name'):
                        user['ContactPersonName'] = sub_sub_prop.text



print(user)


tag:  {http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}srcRef
attrib:  {}
text:  
    
sub_tag:  {http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}Person
sub_attrib:  {'{http://www.opengis.net/gml}id': 'SnowPilot-User-15812'}
sub_text:  
      
sub_sub_tag:  {http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}name
sub_sub_attrib:  {}
sub_sub_text:  katisthebatis
{}


In [16]:
# Weather Conditions

weatherConditions = next(root.iter(common_tag + 'weatherCond'),None)

for prop in weatherConditions.iter():
    print(prop.tag)
    print(prop.text)


{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}weatherCond

        
{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}skyCond
OVC
{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}precipTI
-SN
{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}airTempPres
5
{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}windSpd
C
{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}windDir

          
{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}AspectPosition

            
{http://caaml.org/Schemas/SnowProfileIACS/v6.0.3}position
SE


In [ ]:
    for prop in weatherConditions:
        if prop.tag.endswith('skyCond'):
            pit.snowProfile.weatherConditions.set_skyCond(prop.text)
        if prop.tag.endswith('precipTI'):
            pit.snowProfile.weatherConditions.set_precipTI(prop.text)
        if prop.tag.endswith('airTempPres'):
            temp_val = prop.text
            temp_units = prop.get('uom')
            temp = [float(temp_val), temp_units]
            pit.snowProfile.weatherConditions.set_airTempPres(temp)
        if prop.tag.endswith('windSpeed'):
            windSpeed_val = prop.text
            windSpeed_units = prop.get('uom')
            windSpeed = [float(windSpeed_val), windSpeed_units]
            pit.snowProfile.weatherConditions.set_windSpeed(windSpeed)
        if prop.tag.endswith('windDir'):
            for sub_prop in prop:
                if sub_prop.tag.endswith('position'):
                    pit.snowProfile.weatherConditions.set_windDir(sub_prop.text)